In [37]:
# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install -e .

In [1]:
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import torch
from torch import nn
import torchvision.transforms as transforms

import pytorch_lightning as pl
from torchmetrics.classification import BinaryAccuracy
from pytorch_lightning.callbacks import ModelCheckpoint 
from pytorch_lightning.loggers import TensorBoardLogger

from src.data.project1.dataloader import get_loaders, get_normalization_constants
from src.utils import set_seed

In [39]:
SEED = 0

set_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
accuracy = BinaryAccuracy().to(device)

## Loading model

In [11]:
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=2)

In [12]:
total_params = sum(p.numel() for p in model.parameters())  # Count total parameters
percentage_to_freeze = 0.5  # or 0.2 for 40% or 20% respectively
params_to_freeze = int(percentage_to_freeze * total_params)  # Calculate number of parameters to freeze

frozen_params = 0
non_frozen_params = 0
for param in model.parameters():
    if frozen_params < params_to_freeze:
        param.requires_grad = False  # Freeze the parameter
        frozen_params += param.numel()  # Update the count of frozen parameters
    else:
        non_frozen_params += param.numel()

print(frozen_params)
print(non_frozen_params)

8793672
8758530


In [14]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(f"Layer {name} is frozen.")

Layer conv_stem.weight is frozen.
Layer bn1.weight is frozen.
Layer bn1.bias is frozen.
Layer blocks.0.0.conv_dw.weight is frozen.
Layer blocks.0.0.bn1.weight is frozen.
Layer blocks.0.0.bn1.bias is frozen.
Layer blocks.0.0.se.conv_reduce.weight is frozen.
Layer blocks.0.0.se.conv_reduce.bias is frozen.
Layer blocks.0.0.se.conv_expand.weight is frozen.
Layer blocks.0.0.se.conv_expand.bias is frozen.
Layer blocks.0.0.conv_pw.weight is frozen.
Layer blocks.0.0.bn2.weight is frozen.
Layer blocks.0.0.bn2.bias is frozen.
Layer blocks.0.1.conv_dw.weight is frozen.
Layer blocks.0.1.bn1.weight is frozen.
Layer blocks.0.1.bn1.bias is frozen.
Layer blocks.0.1.se.conv_reduce.weight is frozen.
Layer blocks.0.1.se.conv_reduce.bias is frozen.
Layer blocks.0.1.se.conv_expand.weight is frozen.
Layer blocks.0.1.se.conv_expand.bias is frozen.
Layer blocks.0.1.conv_pw.weight is frozen.
Layer blocks.0.1.bn2.weight is frozen.
Layer blocks.0.1.bn2.bias is frozen.
Layer blocks.1.0.conv_pw.weight is frozen.
L

In [5]:
i = 0
for name, param in model.named_parameters():
    if not not param.requires_grad:
        i += 1
        print(f"Layer {name} is frozen.")

print(i)

Layer blocks.4.1.se.conv_reduce.bias is frozen.
Layer blocks.4.1.se.conv_expand.weight is frozen.
Layer blocks.4.1.se.conv_expand.bias is frozen.
Layer blocks.4.1.conv_pwl.weight is frozen.
Layer blocks.4.1.bn3.weight is frozen.
Layer blocks.4.1.bn3.bias is frozen.
Layer blocks.4.2.conv_pw.weight is frozen.
Layer blocks.4.2.bn1.weight is frozen.
Layer blocks.4.2.bn1.bias is frozen.
Layer blocks.4.2.conv_dw.weight is frozen.
Layer blocks.4.2.bn2.weight is frozen.
Layer blocks.4.2.bn2.bias is frozen.
Layer blocks.4.2.se.conv_reduce.weight is frozen.
Layer blocks.4.2.se.conv_reduce.bias is frozen.
Layer blocks.4.2.se.conv_expand.weight is frozen.
Layer blocks.4.2.se.conv_expand.bias is frozen.
Layer blocks.4.2.conv_pwl.weight is frozen.
Layer blocks.4.2.bn3.weight is frozen.
Layer blocks.4.2.bn3.bias is frozen.
Layer blocks.4.3.conv_pw.weight is frozen.
Layer blocks.4.3.bn1.weight is frozen.
Layer blocks.4.3.bn1.bias is frozen.
Layer blocks.4.3.conv_dw.weight is frozen.
Layer blocks.4.3.b

## Dataloader

In [56]:
ROOT = '/dtu/datasets1/02514/hotdog_nothotdog'
SEED = 0
BATCH_SIZE = 32

# Get normalization constants
train_mean, train_std = get_normalization_constants(root=ROOT, seed=SEED)

# Define transforms for training
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),         # flips "left-right"
    # transforms.RandomVerticalFlip(p=1.0),           # flips "upside-down"
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(60, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=train_mean, 
        std=train_std, 
    )
])

# Define transforms for test and validation
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=train_mean, 
        std=train_std, 
    )
])

# Get data loaders with applied transformations
loaders = get_loaders(
    root=ROOT, 
    batch_size=BATCH_SIZE, 
    seed=SEED, 
    train_transforms=train_transforms, 
    test_transforms=test_transforms, 
    num_workers=24
)

Computing mean of training split...:  47%|▍| 776/1638 [00:02<00:03, 262.89i


KeyboardInterrupt: 

## Output of model

In [ ]:
# Get example batch
batch, targets = next(iter(loaders['train'])) 
out = model(batch)
torch.argmax(nn.functional.softmax(out, dim=1), dim = 1)

tensor([1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 0, 1])

In [ ]:
# torch.nn.functional.one_hot(targets, num_classes=2)

## Model-class (Lightning)

In [ ]:
# Transfer learning model
class HotdogEfficientNet(pl.LightningModule):
    def __init__(self, verbose=False):
        super().__init__()

        # Load model
        self.efficient_net = timm.create_model('efficientnet_b4', pretrained=True, num_classes=2)
        # Freeze weights
        for param in self.efficient_net.parameters():
            param.requires_grad = False
        
        # Require gradient for classification layer
        self.efficient_net.classifier.requires_grad_()


        # Define metrics and loss criterion
        self.criterion = nn.BCEWithLogitsLoss() # nn.BCELoss()
        self.accuracy = BinaryAccuracy().to(self.device)

        # Set up logging option
        self.model_checkpoint = ModelCheckpoint(
            monitor = "val_loss",
            verbose = verbose,
            filename = "{epoch}_{val_loss:.4f}",
        )

    def forward(self,x):
        return self.efficient_net(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = 1e-4)

    def training_step(self, batch, batch_idx):
        # Extract and process input
        x, y = batch
        y = torch.nn.functional.one_hot(y, num_classes=2) 
        y = y.to(torch.float32)

        # Get prediction, loss and accuracy
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)

        # logs metrics for each training_step - [default:True],
        # the average across the epoch, to the progress bar and logger-[default:False]
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True),
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Extract and process input
        x, y = batch
        x, y = x.to(self.device), y.to(self.device)
        y = torch.nn.functional.one_hot(y, num_classes=2) 
        y = y.to(torch.float32) 

        # Get prediction, loss and accuracy
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)

        # logs metrics for each validation_step - [default:False]
        #the average across the epoch - [default:True]
        self.log("val_acc", acc, prog_bar=True, logger=True),
        self.log("val_loss", loss, prog_bar=True, logger=True)

## Training

In [ ]:
SEED = 0
set_seed(SEED)

# Initialize model
HotdogModel = HotdogEfficientNet(
    verbose=False,
)

# Set up logger
tb_logger = TensorBoardLogger(
    save_dir="/work3/s194253/02514/DL-COMVIS/logs/project1",
    version=None,
    name='efficient_net'
),

#CPU:default,GPU:gpus,TPU:tpu_cores
trainer = pl.Trainer(
    devices=1, 
    accelerator="gpu", 
    max_epochs = 100,
    log_every_n_steps=2,
    callbacks=[HotdogModel.model_checkpoint],
    logger=tb_logger,
) 

# Train model
trainer.fit(
    model=HotdogModel,
    train_dataloaders = loaders['train'],
    val_dataloaders = loaders['validation'], 
) 

# manually you can save best checkpoints - 
trainer.save_checkpoint("DL-COMVIS/models/modelshotdog_efficient_net.ckpt")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name          | Type              | Params
----------------------------------------------------
0 | efficient_net | EfficientNet      | 17.6 M
1 | criterion     | BCEWithLogitsLoss | 0     
2 | accuracy      | BinaryAccuracy    | 0     
----------------------------------------------------
3.6 K     Trainable params
17.5 M    Non-trainable params
17.6 M    Total params
70.209    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
